In [1]:
# Correlation coeff, SVD, and decision tree combo
# Super overfitting. Like really bad overfitting

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.decomposition import PCA, TruncatedSVD

import pandas as pd
import numpy as np
import csv
import locale
from scipy.stats import randint
import joblib
locale.setlocale( locale.LC_ALL, '' )

'English_United States.1252'

In [2]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
housing = train_set.copy()

# dropped because way too many null values
housing = housing.drop("PoolQC", axis=1)
housing = housing.drop("Fence", axis=1)
housing = housing.drop("MiscFeature", axis=1)
housing = housing.drop("FireplaceQu", axis=1)
housing = housing.drop("Alley", axis=1)

# dropped because overfitting
housing = housing.drop("Id", axis=1)

housing_labels = pd.DataFrame(train_set["SalePrice"].copy())
ids = test_set["Id"]

corr_matrix = housing.corr(numeric_only=True)
housing = housing.drop("SalePrice", axis=1)
list_col = list(corr_matrix)

for k in range(len(list_col)):
    if corr_matrix["SalePrice"][k] < 0.6:
        housing = housing.drop(f"{list_col[k]}", axis=1)

In [7]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Street         1460 non-null   object
 2   LotShape       1460 non-null   object
 3   LandContour    1460 non-null   object
 4   Utilities      1460 non-null   object
 5   LotConfig      1460 non-null   object
 6   LandSlope      1460 non-null   object
 7   Neighborhood   1460 non-null   object
 8   Condition1     1460 non-null   object
 9   Condition2     1460 non-null   object
 10  BldgType       1460 non-null   object
 11  HouseStyle     1460 non-null   object
 12  OverallQual    1460 non-null   int64 
 13  RoofStyle      1460 non-null   object
 14  RoofMatl       1460 non-null   object
 15  Exterior1st    1460 non-null   object
 16  Exterior2nd    1460 non-null   object
 17  MasVnrType     1452 non-null   object
 18  ExterQual      1460 non-null

In [9]:
text_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))
num_norm_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
preprocess = ColumnTransformer([
    ("text", text_pipeline, make_column_selector(dtype_include=object))],
    remainder=num_norm_pipeline
)

In [11]:
lin_reg = Pipeline([("pre", preprocess), ("pca", TruncatedSVD()), ("reg", DecisionTreeRegressor())])
lin_reg.fit(housing, housing_labels)

housing_predictions = lin_reg.predict(housing)
for k in range(10):
    print(f"Real value: {locale.currency(housing_labels.iloc[k].values[0], grouping=True)}, Prediction: {locale.currency(housing_predictions[k].round(2), grouping=True)}")

Real value: $208,500.00, Prediction: $208,500.00
Real value: $181,500.00, Prediction: $181,500.00
Real value: $223,500.00, Prediction: $223,500.00
Real value: $140,000.00, Prediction: $140,000.00
Real value: $250,000.00, Prediction: $250,000.00
Real value: $143,000.00, Prediction: $143,000.00
Real value: $307,000.00, Prediction: $307,000.00
Real value: $200,000.00, Prediction: $200,000.00
Real value: $129,900.00, Prediction: $129,900.00
Real value: $118,000.00, Prediction: $118,000.00


In [12]:
lin_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
print(lin_rmse)

466.99169311493773


In [14]:
# test_pred = lin_reg.predict(test_set)
# with open('sacreddeer_house_submission_6.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k]])